# Build Document Research - Multi-Agent RAG system


**Objective**: a multi-agent RAG tool designed to help you ask questions about your documents and receive fact-checked, avoid hallucination answers by using reflection pattern asking system to evaluate the answer.

## Install Required Libraries

In [1]:
%%capture
%pip install -q langchain-ibm==0.3.10
%pip install -q langchain==0.3.23
%pip install -q langchain_community==0.3.21 
!pip install langchain-openai==0.3.27
!pip install langgraph==0.6.6
!pip install pygraphviz==1.14

## Import Required Libraries

In [ ]:
# LangGraph modules
from langgraph.graph import StateGraph, END, START

# Type Safety and State Management
from typing import TypedDict, Annotated, List, Literal
from pydantic import BaseModel, Field
import operator
from pprint import pprint

# Graph visualization
from IPython.display import Image, display

# LangChain and LLM 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Initialize the language model - using gpt-4o-mini for cost-effective experimentation
# The Open API was imported outside
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## Process documents and 